In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,auc
from textblob import Word
from nltk.corpus import stopwords

In [2]:
from sklearn.datasets import fetch_20newsgroups
train = fetch_20newsgroups(subset='train', shuffle=True)
train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [3]:
print(type(train))

<class 'sklearn.utils.Bunch'>


In [4]:
print("\n".join(train.data[0].split("\n")[:]))

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [5]:
print("\n".join(train.data[1].split("\n")[:]))

From: guykuo@carson.u.washington.edu (Guy Kuo)
Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
Keywords: SI,acceleration,clock,upgrade
Article-I.D.: shelley.1qvfo9INNc3s
Organization: University of Washington
Lines: 11
NNTP-Posting-Host: carson.u.washington.edu

A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.

Guy Kuo <guykuo@u.washington.edu>



In [6]:
print(train.target_names[train.target[0]])

rec.autos


In [7]:
print(train.target_names[train.target[1]])

comp.sys.mac.hardware


In [8]:
def transformations(dataframe):
    
    # make all character lower
    dataframe['text'] = dataframe['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    
    # removing punctuations
    dataframe['text'] = dataframe['text'].str.replace('[^\w\s]',' ')
    
    #numbers
    dataframe['text'] = dataframe['text'].str.replace('\d','')

    # removing stopwords
    sw = stopwords.words('english')
    dataframe['text'] = dataframe['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    
    # rare characters deleting
    sil = pd.Series(' '.join(dataframe['text']).split()).value_counts()[-1000:]
    dataframe['text'] = dataframe['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sil))
    
    # lemmatization
    
    dataframe['text'] = dataframe['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])) 
    
    return dataframe

In [9]:
df = pd.DataFrame()
df['text'] = pd.DataFrame(train.data)
df['target'] = pd.Series(train.target)
df.head()

,text,target
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14


In [10]:
df = transformations(df)
df.head()

/tmp/ipykernel_22208/2607401895.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe['text'] = dataframe['text'].str.replace('[^\w\s]',' ')
/tmp/ipykernel_22208/2607401895.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe['text'] = dataframe['text'].str.replace('\d','')


,text,target
0,lerxst wam umd edu thing subject car nntp post...,7
1,guykuo carson u washington edu guy kuo subject...,4
2,twillis ec ecn purdue edu thomas e willis subj...,4
3,jgreen amber joe green subject weitek p organi...,1
4,jcm head cfa harvard edu jonathan mcdowell sub...,14


In [11]:
train_df = df['text'].values.tolist()

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

processor = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

In [17]:
processed = processor.fit(train.data,train.target)
processed.score(train.data,train.target)

0.9326498143892522

In [18]:
test = fetch_20newsgroups(subset='test', shuffle=True)
predicted = processed.predict(test.data)
print(np.mean(predicted == test.target))

0.7738980350504514


In [19]:
processed = processor.fit(train_df,train.target)
processed.score(train_df,train.target)

0.9513876613045784

In [20]:
test = fetch_20newsgroups(subset='test', shuffle=True)
predicted = processed.predict(test.data)
print(np.mean(predicted == test.target))

0.7873074880509825
